# Cuaderno 28: Implementando algoritmos de plano cortante
## El problema de ordenamiento lineal 
## Linear Ordering Problem (LOP)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$, sin lazos; y,
* un vector $w \in \ZZ^{A}$ de pesos asociados a los arcos de $D$.

El *problema de ordenamiento lineal (Linear Ordering Problem, LOP)* consiste en encontrar un conjunto de arcos con las siguientes propiedades:
* entre cada par de nodos debe seleccionarse exactamente uno de los dos arcos (antiparalelos) posibles;
* los arcos seleccionados no deben formar circuitos; y,
* la suma de los pesos de los arcos seleccionados debe ser máxima.

Notar que la segunda condición equivale a requerir que los arcos seleccionados no formen circuitos de longitud 3, es decir, triángulos orientados.

Empleando variables binarias $x_{ij}$ para indicar selección de arcos, LOP puede formularse como el siguiente programa lineal entero:

\begin{align*}
\max &\sum_{(i,j) \in A} w_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
& x_{ij} + x_{ji} = 1, \quad \forall i \neq j \in V,\\
& x_{ij} + x_{jk} + x_{ki} \leq 2, \forall i, j, k \in V \mbox{ distintos entre sí},\\
& x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A.
\end{align*}

Vamos a implementar este modelo usando la interfaz Python de Gurobi. Colocaremos las restricciones de arcos antiparalelos de la manera usual en el modelo, pero agregaremos las restricciones de eliminación de triángulos de manera dinámica durante el proceso de solución.


Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {-10,..,10} para los pesos de los arcos. 

In [1]:
from gurobipy import *
import random as rm
import math 

# Numero de nodos del grafo
n = 8

# Nodos del grafo: {1,..,n}
V = tuplelist(range(1,n+1))

# los pesos son valores aleatorios en el rango {-10, 10}
w = tupledict({})
for i in V:
    for j in V:
        # evitar lazos
        if i != j:
            w[i,j]= rm.randint(-10, 10)
A = w.keys()

print('*** Matriz de pesos ***')
print(''.join(["{:3}   ".format(i) for i in V]))
print('-'*60)
for i in V:
    L =[]
    for j in V:
        if i!=j:
            L.append("{:3}   ".format(w[i,j]))
        else:
            L.append("---   ")
    print(''.join(L))
        


*** Matriz de pesos ***
  1     2     3     4     5     6     7     8   
------------------------------------------------------------
---    -4    -3    -3     2    -6    -2     7   
 -4   ---    -2   -10     7    -9    10    -1   
 10     5   ---     1    -2    -9     1    -2   
 -8     4     1   ---     0     4    -7     8   
 -8    -6    -4     5   ---     5    -4     9   
 10    -5     8   -10    -4   ---     9    -8   
 -6     9     2    -9     0     1   ---     5   
 -8     1     3   -10    -6     3    -6   ---   


Definimos el objeto modelo, las variables y la función objetivo. Las asignaciones `m._x = x` y `m._V = V` serán utilizadas más adelante.

In [2]:
# Crear el objeto modelo
m = Model('LOP')

# Variables de seleccion de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)
m._x = x
m._V = V

# Crear la funcion objetivo
m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)


Añadimos las restricciones de selección de arcos antiparalelos, que seleccionan exactamente un arco entre cada par de nodos:

In [3]:
# Restricciones de seleccion de arcos antiparalelos
m.addConstrs((x[i,j] + x[j,i]==1 for i,j in A if i < j), "antiparalelos")


{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 7): <gurobi.Constr *Awaiting Model Update*>,
 (3, 8): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 6): <gurobi.Constr *Awaiting Model Update*>,


### Restricciones lazy y funciones callback
Las restricciones de eliminación de triángulos se implementarán como restricciones tipo *lazy*. Estas restricciones se agregan dinámicamente al modelo conforme son requeridas. Para ello, es necesario definir una función tipo **callback**.

Una función tipo callback es una función diseñada para ser llamada por el solver Gurobi durante el proceso de solución del modelo. Esta función puede tener cualquier nombre, pero debe recibir obligatoriamente dos parámetros: `model` y `where`. La función debe en primer lugar examinar el valor de `where` para determinar desde qué parte del proceso de solución ha sido llamada y tomar la acción que sea correspondiente. Esto se hace comparando este parámetro contra algunos *códigos callback*. En nuestro ejemplo, el código `GRB.Callback.MIPSOL` indica que se acaba de encontrar una nueva solución entera que satisface las demás restricciones del problema. En el manual de referencia puede consultarse la lista completa de [códigos callback](https://www-proxy.gurobi.com/documentation/8.1/refman/callback_codes.html#sec:CallbackCodes).

El parámetro `model` proporciona acceso a las variables y métodos del objeto modelo. Esto es útil para acceder a ciertas funciones como `cbGetSolution`, que permite recuperar la solución actual; o `cbLazy` que permite agregar dinámicamente una restricción lazy al modelo. También se usa para tener acceso a variables y cualquier otra información del modelo. Para ello, en el programa principal, la información a compartir debe incluirse como una variable del objeto modelo, cuyo nombre debe empezar con el símbolo `_`. En nuestro ejemplo, empleamos `_x`para acceder de las variables de selección de arcos y `_V`para acceder a la lista con los vértices.

En el ejemplo, la función callback se activa únicamente cuando se ha encontrado una nueva solución entera que satisface todas las desigualdades de arcos antiparalelos. Llamando a `cbGetSolution(model._x)` se recuperan los valores de las variables $x_{ij}$ dentro de la solución actual y se almacenan en `vx`. Luego se itera sobre todos los conjuntos de tres nodos y se verifica que estos valores satisfagan las dos desigualdades triangulares posibles. Cada vez que se encuentra un triángulo donde la suma de los valores de las variables supera 2, se agrega la desigualdad correspondiente llamando a la función `cbLazy`.

In [4]:
# Definir funcion callback
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar si existe desigualdad triangular que sea violada
        for i in model._V:
            for j in model._V:
                if i>=j: continue
                for k in model._V:
                    if j>=k: continue
                    # Iteramos sobre todos los nodos i < j < k, y verificamos las dos restricciones posibles
                    if vx[i, j] + vx[j, k] + vx[k, i] > 2:
                        model.cbLazy(model._x[i, j] + model._x[j, k] 
                                                    + model._x[k, i] <=2)
                    if vx[i, k] + vx[k, j] + vx[j, i] > 2:
                        model.cbLazy(model._x[i, k] + model._x[k, j] 
                                           + model._x[j, i] <=2)
        

Para poder utilizar restricciones tipo *lazy* en un modelo, debe fijarse el parámetro `LazyConstraints` al valor de 1.

In [5]:
# Configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1


Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


Fijamos un tiempo límite de 180 segundos y una tolerancia para la brecha de optimalidad del 15%:

In [6]:
# Terminar al alcanzar un Gap del 15%
m.Params.MIPGap = 0.2

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180


Changed value of parameter MIPGap to 0.2
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


Escribimos el modelo a un archivo de texto. Notar que el modelo inicial contiene solamente las restricciones de selección de arcos antiparalelos.

In [7]:
# Escribir el modelo a un archivo
m.write('LOP-lazy.lp')


Finalmente, resolvemos el modelo y mostramos la solución. Al llamar a `optimize` es necesario pasar el nombre de la función *callback* como un parámetro.

In [8]:
# Calcular la solucion optima
m.optimize(mycallback)

# Escribir la solucion
if ((m.status == GRB.Status.OPTIMAL) or 
   (m.status == GRB.Status.SUBOPTIMAL)):
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nOrdenamiento optimo:')
    for i,j in A:
        if vx[i,j] > 0:
            print('{} -> {}'.format(i, j))


Optimize a model with 28 rows, 56 columns and 56 nonzeros
Variable types: 0 continuous, 56 integer (56 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -4.0000000
Presolve removed 28 rows and 28 columns
Presolve time: 0.09s
Presolved: 0 rows, 28 columns, 0 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)

Root relaxation: objective 9.500000e+01, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      57.0000000   57.00000  0.00%     -    0s

Cutting planes:
  Lazy constraints: 19

Explored 0 nodes (11 simplex iterations) in 0.24 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 57 -4 

Optimal solution found (tolerance 2.00e-01)
Best objective

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [10]:
# Implementación de modelos lineales enteros
# Problema de ordenamiento lineal
# Formulación usando restricciones lazy para las desigualdades de eliminación de triángulos

# Luis M. Torres (EPN 2019)

from gurobipy import *
import random as rm

# Definir funcion callback
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar si existe desigualdad triangular que sea violada
        for i in model._V:
            for j in model._V:
                if i>=j: continue
                for k in model._V:
                    if j>=k: continue
                    # Iteramos sobre todos los nodos i < j < k, y verificamos las dos restricciones posibles
                    if vx[i, j] + vx[j, k] + vx[k, i] > 2:
                        model.cbLazy(model._x[i, j] + model._x[j, k] + model._x[k, i] <=2)
                    if vx[i, k] + vx[k, j] + vx[j, i] > 2:
                        model.cbLazy(model._x[i, k] + model._x[k, j] + model._x[j, i] <=2)


# Numero de nodos del grafo
n = 50

# Nodos del grafo: {1,..,n}
V = tuplelist(range(1,n+1))

# los pesos son valores aleatorios en el rango {-10, 10}
w = tupledict({})
for i in V:
    for j in V:
        # evitar lazos
        if i != j:
            w[i,j]= rm.randint(-10, 10)
A = w.keys()

# Escribir la matriz con los datos ingresados
print('*** Matriz de pesos ***')
print(''.join(["{:3}   ".format(i) for i in V]))
print('-'*60)
for i in V:
    L =[]
    for j in V:
        if i!=j:
            L.append("{:3}   ".format(w[i,j]))
        else:
            L.append("---   ")
    print(''.join(L))


try:
    # Crear el objeto modelo
    m = Model('LOP')

    # Variables de seleccion de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)

    # Crear variables en el objeto modelo para el acceso 
    # a las variables de selección de arcos y al conjunto de nodos
    m._x = x
    m._V = V

    # Restricciones de seleccion de arcos antiparalelos
    m.addConstrs((x[i,j] + x[j,i]==1 for i,j in A if i < j), "antiparalelos")

    # Escribir el modelo a un archivo
    m.write('LOP-lazy.lp')

    # Configurar Gurobi para usar restricciones lazy
    m.Params.LazyConstraints = 1

    # Terminar al alcanzar un Gap del 15%
    m.Params.MIPGap = 0.2

    # Terminar luego de 180 segundos
    m.Params.TimeLimit = 180

    # Iniciar algoritmo de solucion
    m.optimize(mycallback)

    # Escribir la solucion
    if ((m.status == GRB.Status.OPTIMAL) or 
       (m.status == GRB.Status.SUBOPTIMAL)):
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nOrdenamiento optimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
                
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

*** Matriz de pesos ***
  1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39    40    41    42    43    44    45    46    47    48    49    50   
------------------------------------------------------------
---     4    -1     5    -6     9    -3    -9     2     9     9    -7    -2     5    -3    -2     5     7     5    -4    -4     1     2     2   -10     0   -10     9    -3     7     8     6     3    10    -3    -3    -2    -7     5    -6     7     8    -7     7    -6     9    -9     1     6     3   
  9   ---    -4   -10     9    -4     2    -2    -8     2    -9     2    -4     7    -1    -3    -4    -3     0     2     9     7   -10     0    -4    -1     4    10    -3    -4     6     8    -8     3    -2     8    -8    -4     5     7     5    -1     1    -5    -9     0     7     8     7    -2   
 -2     1   

   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter MIPGap to 0.2
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 1225 rows, 2450 columns and 2450 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -402.0000000
Presolve removed 1225 rows and 1225 columns
Presolve time: 0.00s
Presolved: 0 rows, 1225 columns, 0 nonzeros
Variable types: 0 continuous, 1225 integer (1225 binary)

Root relaxation: objective 3.928000e+03, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1504.00000    0  693 -402.00000 